# The relationship between the Human Development Index and the prevalance of mental health disorders

## Research question

The aim of this analysis is to investigate the relationship between country-level socioeconomic status and the prevalence of mental health disorders. Studies have suggested that mental health is worse in relatively highly developed countries [(Barbalat & Franck, 2020)](https://bmjopen.bmj.com/content/bmjopen/10/4/e035055.full.pdf). A country's level of human development can be measured using the human development index (HDI).

The HDI is a metric designed to quantify a country's socieeconomic status by using key dimensions of human development. The index is calculated using four key metrics: (1) life expectancy at birth - to asses a long and healthy life, (2) expected years of schooling - to asses access to knowledge of the younger generation, (3) average years of schooling - to assess access to knowledge of the older generation and (4) gross national income (GNI) per capita - to asses the standard of living. Each of these metrics is normalized to an index value and aggregated into the HDI using the following formula [(2020 HDR Technical Notes)](https://hdr.undp.org/system/files/documents/technical-notes-calculating-human-development-indices.pdf):

$HDI = (I_{Health} * I_{Education} * I_{Income})^\frac{1}{3}$

The HDI dataset contains HDI values from 1990 to 2021 The 'Global Trends in Mental Health Disorder' file contains the prevalence expressed in percentages for schizophrenia, bipolar disorder, eating disorders, anxiety disorders, drug use disorders, depression and alcohol use disorders from 1990 to 2017. The data sources are given at the end of the notebook under "References". 

To determine whether there is a statistical relationship between the prevalence of the mentioned mental health disorders, linear regression and correlation analysis is performed. Linear regression estimates whether there is a relationship between a response variable (y) and explanatory variables (x) by plotting the best fit line using the "least squares" method. The R-squared value indicates the strength of the relationship between the model and the dependent variables (HDI in this case).  Spearman's rank correlation summarises the strenth and direction of the relationship giving a score from 0 to 1.

### Loading needed libraries

In [72]:
# Loading needed libraries
import yaml
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import json
import matplotlib.pyplot as mpl
import pylab as plt
import re

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar, Label,FactorRange
from bokeh.plotting import figure,show
from bokeh.palettes import brewer

import panel as pn
import panel.widgets as pnw
pn.extension()

import scipy
import scipy.stats as stats
from scipy.stats import norm

from sklearn import linear_model

### Defining  functions

#### Functions needed for data loading, cleaning and inspection

In [73]:
# Getting data location from configuration file
def get_config():
    with open("config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config
    
# Printing information about the datasets
def data_inspection(df):
    
    # Displaying number of columns and rows, datatypes, missing values
    print(f'Dataset contains {df.shape[0]} rows and {df.shape[1]} columns. \n')
    print(f'Datatypes: \n{df.dtypes} \n')
    print(f'Missing data per column: \n{df.isnull().sum()}')

    # Displaying number of countries and years included
    print(f'\nNumber of countries included: {len(df.Entity.unique())}')
    print(f'Number of years included: {len(df.Year.unique())}')
    
    # Displaying timespan of included years=
    print(f'Timespan: {df.Year.min()}-{df.Year.max()}')
    

# Reformating the mental health dataset
def reformat_mh():
    # Drop index columns
    df = mh_raw.drop('index', axis='columns')

    # Rename columns
    for column in df.columns:
        if re.search('\(.\)', column):
            name = column.replace(" (%)", "")
            df.rename(columns={column:name}, inplace=True)

    # Drop missing values
    df = df.dropna()

    # Changing datatypes
    df['Year'] = df['Year'].astype(int)
    df['Schizophrenia'] = df['Schizophrenia'].astype(float)
    df['Bipolar disorder'] = df['Bipolar disorder'].astype(float)
    df['Eating disorders'] = df['Eating disorders'].astype(float)
    
    return df

# Replacing mulitple country names to match between datasets
def correct_countries(df, dict):
    for x in dict.items():
        df['Entity'] = df['Entity'].replace(x[0], x[1])

    return df

#### Functions needed for data visualisation

In [74]:
def plot_histogram(variable, year):
    
    # Setting plot parameters
    p = figure(width=800, height=600, toolbar_location=None,
           title="Distribution of the selected variable:")
    p.xaxis.axis_label = "Distribution"
    p.yaxis.axis_label = "Count"
    
    # Selecting HDI or MH dataset
    if variable == "Human Development Index":
        df = hdi_clean

    else:
        df = mh_clean
        
    # Plotting per year
    df = df[df['Year'] == year]
    
    # Setting x
    begin = min(df[variable])
    end = max(df[variable])
    x = np.linspace(begin, end, 30)
    
    # Plotting histogram
    hist, edges = np.histogram(df[variable], density=True, bins=x)
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
             fill_color="#feb24c", fill_alpha = 0.8, line_color="white")
    
    return p

In [75]:
def scatter_plot(year,variable):
    
    df = mh_hdi

    # Plotting per year
    df = df[df['Year'] == year]
    
    source = ColumnDataSource(df)
    
    # Setting plot parameters
    p = figure(width=800, height=600, title="Scatterplot HDI and mental health prevalence")
    p.xaxis.axis_label = "Human Development Index (HDI)"
    p.yaxis.axis_label = "Prevalance (%)"
    
    # Scatter plot
    p.circle("Human Development Index",variable, 
                 size=9, color="color", alpha=0.8, 
                 legend_field = "HDI scale", source=source)
    
    # Customizing legend
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    p.legend.background_fill_color = "grey"
    p.legend.background_fill_alpha = 0.15
    p.legend.label_text_font_size = "10.5pt"
    p.legend.label_text_font_style = "italic"
    
    return p

In [76]:
def simple_linear_regression(year,variable):
    
    # Plotting per year
    df = mh_hdi[mh_hdi['Year'] == year]
    
    
    # Setting plot parameters
    source = ColumnDataSource(df)
    p = figure(width=800, height=600,
              title="Linear Regression")
    p.xaxis.axis_label = "Human Development Index (HDI)"
    p.yaxis.axis_label = "Prevalance (%)"
    
    # Scatter plot
    p.circle("Human Development Index",variable, size=9, color="color", alpha=0.8, 
                 legend_field = "HDI scale", source=source)
    
    # Simple linear regression
    x = df["Human Development Index"]
    y = df[variable]
    
    # Initializing linear model
    regression_model = linear_model.LinearRegression()

    # Train the model using the data
    regression_model.fit(X = pd.DataFrame(df["Human Development Index"]), 
                     y = df[variable])
    
    # Trained model y-intercept
    intercept = regression_model.intercept_

    # Trained model coefficients
    slope = regression_model.coef_[0]
    
    # Getting linear prediction
    y_predicted = [slope*i + intercept  for i in x]
    
    # Getting model score
    score = regression_model.score(X = pd.DataFrame(df["Human Development Index"]), 
                     y = df[variable])
    
    # Adding model score (R2) to plot
    a = max(df["Human Development Index"]) - 0.2
    b = min(df[variable])
    
    mytext = Label(x=a, y=b, text = f'y = {round(slope,3)}x + {str(round(intercept,3))}   R\N{SUPERSCRIPT TWO} = {round(score,3)}', 
                       text_color = "crimson", text_font_size = "10pt")
    
    # Linear regression
    p.line(x,y_predicted,color='crimson', alpha = 0.7, line_width = 2, line_dash = 'dotted',)
    
    # Adding text and legends
    p.add_layout(mytext)
    
    # Customizing legend
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    p.legend.background_fill_color = "grey"
    p.legend.background_fill_alpha = 0.15
    p.legend.label_text_font_size = "10.5pt"
    p.legend.label_text_font_style = "italic"
    
    return p 

In [77]:
def polynomial_regression(year,variable):
    
    # Plotting per year
    df = mh_hdi[mh_hdi['Year'] == year]
    
    # Setting plot parameters
    source = ColumnDataSource(df)
    p = figure(width=800, height=600,
              title="Polynomial Regression")
    p.xaxis.axis_label = "Human Development Index (HDI)"
    p.yaxis.axis_label = "Prevalance (%)"
    
    # Scatter plot
    p.circle("Human Development Index",variable, size=9, color="color", alpha=0.8, 
                 legend_field = "HDI scale", source=source)
    
    # Setting x and y
    x = df["Human Development Index"]
    y = df[variable]
        
    # Initializing linear model
    poly_model = linear_model.LinearRegression()

    # Dataframe of predictor variables
    predictors = pd.DataFrame([x, x**2]).T
    
    # Train model
    poly_model.fit(X = predictors, y = df[variable])
    
    # Trained model y-intercept
    intercept = poly_model.intercept_

    # Trained model coefficients
    slope = poly_model.coef_
    
    # Getting model score
    score = poly_model.score(X = predictors, 
                     y = y)
    
    # set curve range
    poly_line_range = np.arange(min(x), max(x), 0.05)
    
    # Get first and second order predictors from range
    poly_predictors = pd.DataFrame([poly_line_range,
                               poly_line_range**2]).T
    
    # Get corresponding y values from the model
    y_values = poly_model.predict(X = poly_predictors)
    
    # Adding model score (R2) to plot
    a = max(df["Human Development Index"]) - 0.25
    b = min(df[variable]) 
    text = f'y = {str(round(slope[1],3))}x\N{SUPERSCRIPT TWO} + {str(round(slope[0],3))}x + {(round(intercept,3))}   R\N{SUPERSCRIPT TWO} = {round(score,3)}'
    
    mytext = Label(x=a, y=b, text = text, 
                       text_color = "crimson", text_font_size = "10pt")
    
    # Linear regression
    p.line(poly_line_range, y_values, color='crimson', alpha = 0.7, line_width = 2, line_dash = 'dashed')
    
    p.add_layout(mytext)
           
    # Customizing legend
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    p.legend.background_fill_color = "grey"
    p.legend.background_fill_alpha = 0.15
    p.legend.label_text_font_size = "10.5pt"
    p.legend.label_text_font_style = "italic"
    
    return p 

In [78]:
def correlation_barplot(year):
    
    df = mh_hdi[mh_hdi['Year'] == year]
    spearman = []
    fill_color = []
    
    variables =  list(mh_clean.columns[3:]) 
    for x in variables:
    
        correlation, p = scipy.stats.spearmanr(df[x], df['Human Development Index'])
        spearman.append(correlation)

        if p <= 0.05:
            fill_color.append("#f03b20")

        else:
            fill_color.append("#800026")

    # plotting    
    p = figure(title="Correlation between the prevalence of mental disorders and the HDI",
                toolbar_location=None, tools="",
                y_range=FactorRange(factors=variables),
                width = 800, height = 600)
    p.xaxis.axis_label = "Spearman's correlation coefficient"

    p.hbar(y=variables, right = spearman, height=0.5, color=fill_color, fill_alpha = 0.8, 
           line_color = "black")
    

    return p

## 1. Data loading

To run this analysis download the following files: [human-development-index.csv](https://ourworldindata.org/human-development-index), [Mental health Depression disorder Data.csv](https://www.kaggle.com/datasets/thedevastator/uncover-global-trends-in-mental-health-disorder?resource=download) and store in 'Data' folder. 

In [ ]:
config = get_config()

# Human development index data
d1 = (config['dataset1'])
hdi_raw = pd.read_csv(d1)

# Mental health disorder data
d2 = (config['dataset2'])
mh_raw = pd.read_csv(d2)

## 2. Data inspection 

The data is inspected for structure and format. Column and row number, column datatypes and missing data is assesed for each dataset. 

##### Mental health disorders

In [80]:
# Viewing dataset
mh_raw.head()

,index,Entity,Code,Year,Schizophrenia (%),Bipolar disorder (%),Eating disorders (%),Anxiety disorders (%),Drug use disorders (%),Depression (%),Alcohol use disorders (%)
0,0,Afghanistan,AFG,1990,0.16056,0.697779,0.101855,4.828830,1.677082,4.071831,0.672404
1,1,Afghanistan,AFG,1991,0.160312,0.697961,0.099313,4.829740,1.684746,4.079531,0.671768
2,2,Afghanistan,AFG,1992,0.160135,0.698107,0.096692,4.831108,1.694334,4.088358,0.670644
3,3,Afghanistan,AFG,1993,0.160037,0.698257,0.094336,4.830864,1.705320,4.096190,0.669738
4,4,Afghanistan,AFG,1994,0.160022,0.698469,0.092439,4.829423,1.716069,4.099582,0.669260


In [81]:
# Inspecting dataframe, datatypes, column and row counts and missing values
data_inspection(mh_raw)

Dataset contains 108553 rows and 11 columns. 

Datatypes: 
index                          int64
Entity                        object
Code                          object
Year                          object
Schizophrenia (%)             object
Bipolar disorder (%)          object
Eating disorders (%)          object
Anxiety disorders (%)        float64
Drug use disorders (%)       float64
Depression (%)               float64
Alcohol use disorders (%)    float64
dtype: object 

Missing data per column: 
index                             0
Entity                            0
Code                           5412
Year                              0
Schizophrenia (%)             82678
Bipolar disorder (%)          89147
Eating disorders (%)           8317
Anxiety disorders (%)        102085
Drug use disorders (%)       102085
Depression (%)               102085
Alcohol use disorders (%)    102085
dtype: int64

Number of countries included: 276
Number of years included: 259
Timespan: 0-Year


#### Observations: 

1. There is a lot of missing data. This needs to be filtered out and checked if remaining data is still sufficient and if remaining countries correspond to the HDI dataset. 

2. Several of columns need to be converted from object to the float datatype. 

3. The dataset includes an unexpectedly high number of years.This needs to be examined further

In [82]:
# Checking time span 
print(mh_raw['Year'].unique())

['1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999'
 '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009'
 '2010' '2011' '2012' '2013' '2014' '2015' '2016' '2017' 'Year' '1800'
 '1801' '1802' '1803' '1804' '1805' '1806' '1807' '1808' '1809' '1810'
 '1811' '1812' '1813' '1814' '1815' '1816' '1817' '1818' '1819' '1820'
 '1821' '1822' '1823' '1824' '1825' '1826' '1827' '1828' '1829' '1830'
 '1831' '1832' '1833' '1834' '1835' '1836' '1837' '1838' '1839' '1840'
 '1841' '1842' '1843' '1844' '1845' '1846' '1847' '1848' '1849' '1850'
 '1851' '1852' '1853' '1854' '1855' '1856' '1857' '1858' '1859' '1860'
 '1861' '1862' '1863' '1864' '1865' '1866' '1867' '1868' '1869' '1870'
 '1871' '1872' '1873' '1874' '1875' '1876' '1877' '1878' '1879' '1880'
 '1881' '1882' '1883' '1884' '1885' '1886' '1887' '1888' '1889' '1890'
 '1891' '1892' '1893' '1894' '1895' '1896' '1897' '1898' '1899' '1900'
 '1901' '1902' '1903' '1904' '1905' '1906' '1907' '1908' '1909' '1910'
 '1911

The mh dataset currently contains years that go too far back (including prehistorical data).This can also factor into the high number of missing data. The dataset needs to be filtered to only contain data from the years 1990 to 2017.

##### Human development index

In [83]:
# Viewing the dataframe
hdi_raw.head()

,Entity,Code,Year,Human Development Index
0,Afghanistan,AFG,1990,0.273
1,Afghanistan,AFG,1991,0.279
2,Afghanistan,AFG,1992,0.287
3,Afghanistan,AFG,1993,0.297
4,Afghanistan,AFG,1994,0.292


In [84]:
# Data inspection
data_inspection(hdi_raw)

Dataset contains 5923 rows and 4 columns. 

Datatypes: 
Entity                      object
Code                        object
Year                         int64
Human Development Index    float64
dtype: object 

Missing data per column: 
Entity                       0
Code                       320
Year                         0
Human Development Index      0
dtype: int64

Number of countries included: 202
Number of years included: 32
Timespan: 1990-2021


#### Observations: 

1. There is missing data in the 'Code' column. However, 'Entity' has no missing data. Since these column essentialy contain the same information, 'Entity' can be used to identity countries instead of the country code.

2. Datatypes are correct.

3. Contains less countries than the other dataset, but includes a longer timespan (from 1990 to 2021). This, however, will be filtered later on when merging the dataset to only include data up until 2017.


## 3. Data Wrangling

### 3a. Data cleaning

In this step missing data is removed and datatypes are converted to the correct types. The MH and HDI The MH and HDI dataset have some differences in country names. These need to be corrected before merging the datasets.



##### Mental health disorders

In [85]:
# Removing missing data, renaming columns and converting datatypes
mh_clean = reformat_mh()

In [86]:
# Creating dictionary with country names that need to be replaced
dict1 = {'Czech Republic':'Czechia','Macedonia':'North Macedonia'}
dict2 = {'Serbia':'Republic of Serbia', 'Democratic Republic of Congo':'Democratic Republic of the Congo', 'Bahamas':'The Bahamas',
    'Tanzania':'United Republic of Tanzania', 'United States':'United States of America',  'Timor':'East Timor', "Cote d'Ivoire":'Ivory Coast',
    'Congo':'Republic of the Congo'}

# Correcting country names 
mh_clean = correct_countries(mh_clean, dict1)
mh_clean = correct_countries(mh_clean, dict2)

##### Human Development Index

In [87]:
# Renaming country names to match other dataframes
hdi_clean = correct_countries(hdi_raw, dict2)

### 3b. Combining data

For further data investigation, visualisation and statistical analysis, the mental health and HDI datasets are combined on entity and year.

In [88]:
# Merging HDI with mental disorders
mh_hdi = pd.merge(mh_clean, hdi_clean, on=['Entity', 'Year', 'Code'])

## 4. Data Exploration

To determine which statistical analyses are applicable to the research question the distribution of the datasets is visualised using a histogram. To visualise a possible relationship between mental health prevalence and the HDI, these two variables are plotted in a scatter plot. The HDI van be split into 4 catagories:

1. Low (< 0.550)
2. Medium (0.550-0.699)
3. High (0.700-0.799)
4. Very high (>= 0.800)

The spread of these catagories for each mental health disorder is also examined in the scatted plot

In [89]:
# Adding scale list to dataframe for visualisation 
scale_list = []
for x in mh_hdi['Human Development Index']:
    
    if x >= 0.8:
        scale = "Very high (> 0.800)"
    elif x > 0.7 and x < 0.799:
        scale = "High (0.700-0.799)" 
    elif x > 0.55 and x < 0.699:
        scale = "Medium (0.550-0.699)"
    elif x < 0.55:
        scale = "Low (< 0.550)"
        
    scale_list.append(scale)
    
mh_hdi['HDI scale'] = scale_list

# Adding color for each HDI scale
colors = {'Very high (> 0.800)' : '#0c2c84','High (0.700-0.799)' : '#225ea8',
            'Medium (0.550-0.699)' : '#1d91c0', 'Low (< 0.550)' : '#41b6c4'}

mh_hdi["color"] = mh_hdi["HDI scale"].apply(lambda c: colors[c])

In [90]:
# Widgets for plotting
# Year slider
years = pn.widgets.IntSlider(name='Year', start=1990,end=2017)

# List of all variables (disorders + HDI)
variable_list = list(mh_clean.columns[3:]) + ["Human Development Index"]
variables = pn.widgets.RadioButtonGroup(name='Variables',options=variable_list,button_type='success')

# List of mental disorders
disorder_list =  list(mh_clean.columns[3:]) 
disorders = pn.widgets.RadioButtonGroup(name='Disorder',options=disorder_list,button_type='success')

### 4a. Histogram

In [91]:
# Plotting interactive histogram
histogram = pn.interact(plot_histogram, variable = variables, year = years)

# Displaying panel output in notebook
pn.extension()
pn.Row(histogram)

Row
    [0] Column
        [0] Column
            [0] RadioButtonGroup(button_type='success', name='Variables', options=['Schizophrenia', ...], value='Schizophrenia')
            [1] IntSlider(end=2017, name='Year', start=1990, value=1990)
        [1] Row
            [0] Bokeh(Figure, name='interactive01936')

None of the variables appear to be following a clear normal distribution. For further statistical analysis a normal distribution is not assumed. 

### 4b Scatter plot

In [92]:
pn.Row(mh)

Row
    [0] Column
        [0] Column
            [0] IntSlider(end=2017, name='Year', start=1990, value=1990)
            [1] RadioButtonGroup(button_type='success', name='Disorder', options=['Schizophrenia', ...], value='Schizophrenia')
        [1] Row
            [0] Bokeh(Figure, name='interactive01946')

In general, there appears to be a postive correlation between mental health prevalence and human development index. As the humand development index increases, so does mental health prevalance. This relationship seems to be the strongest for eating disorders, schizophrenia and bipolar disorder (respectively). The relationship appears the weakest for alcohol use disorders, drug use disorders and depression. 

## 5. Statistical analysis

To determine whether the observed positive relationship holds any statistical significance, linear regression and correlation analysis are performed. 

### 5a. Simple linear regression

Linear regression estimates whether there is a relationship between a response variable (y) and explanatory variables (x) by plotting the best fit line. The R-squared value indicated how much of the variation is explained by x. 

In [93]:
# Simple linear regression with HDI as predictor (x) and prevalance of mental disorder as outcome (y)
linear_regression = pn.interact(simple_linear_regression, year=years, variable = disorders)
pn.Row(linear_regression)

Row
    [0] Column
        [0] Column
            [0] IntSlider(end=2017, name='Year', start=1990, value=1990)
            [1] RadioButtonGroup(button_type='success', name='Disorder', options=['Schizophrenia', ...], value='Schizophrenia')
        [1] Row
            [0] Bokeh(Figure, name='interactive01956')

There appears to be a positive correlation between HDI and all the variables except for depression and alcohol disorders. Depression appears to have a small negative relationship to the HDI. The linear model does not appear to fit the data for each variable. Next, a polynomial regression model is explored for a better fit.

### 5b. Polynomial regression

In [ ]:
# Polynomial linear regression
polynomial_regression = pn.interact(polynomial_regression, year=years, variable = disorders)

In [95]:
pn.Row(polynomial_regression)

Row
    [0] Column
        [0] Column
            [0] IntSlider(end=2017, name='Year', start=1990, value=1990)
            [1] RadioButtonGroup(button_type='success', name='Disorder', options=['Schizophrenia', ...], value='Schizophrenia')
        [1] Row
            [0] Bokeh(Figure, name='interactive01966')

The polynomial regression model appears to be a better fit for the data. This is demonstrated by the increase in R-squared for all variables. Eating disorer prevalance demonstrates the highest R-squared, with up to 70% of the variation in prevalance being explained by the HDI. The depression prevalance also demonstrated an increase in R and now shows a positive correlation with up to 13% of the variation being explained by the HDI. 

### 5c. Spearman's correlation test

To determine any statistical correlation an Spearman rank-order correlation coefficient test is performed. This test can be applied to data that does not meet the assumptions of a normal distribution. This test determines the statistical strength and direction of the association between two variables. Insignificant results (p > 0.05) are shown in dark red while red bars show a significant association (p < 0.05). 

In [96]:
correlation = pn.interact(correlation_barplot, year=years)
pn.Row(correlation)

Row
    [0] Column
        [0] Column
            [0] IntSlider(end=2017, name='Year', start=1990, value=1990)
        [1] Row
            [0] Bokeh(Figure, name='interactive01976')

Association between the HDI and mental health prevelance fluctuates over the years. There is a significant positive association between all factor, for all years for all the disorders except alcohol use disorders and depression. Eating disorders, as expected, demonstrate the highest statistical association. Depression demonstrates an insignificant negative association for the majority of the years. For alcohol use disorders there is a significant positive association for some of the years.

## Data visualisation: Dashboard

In [ ]:
import panel as pn
import numpy as np
import holoviews as hv

pn.extension()

# creating widgets
year = pn.widgets.IntSlider(name='Year', start=1990,end=2017)


template = pn.template.FastListTemplate(
    theme = 'default',
    accent_base_color="#b33e56",
    header_background="#bd0026",
    title = 'The Human Development Index (HDI) and the Prevalance of Mental Health Disorders',
    sidebar =[pn.pane.Markdown("## Are Mental Health Disorders More Prevalent in More Developed Countries?"),
              pn.pane.PNG('Data/world-map.png', sizing_mode='scale_both'),
              pn.pane.Markdown("Studies have suggested that mental health is worse in relatively highly developed countries. This analysis aims to investigate the relationship between country-level socioeconomic status (quantified by the Human Development Index) and the prevalence of mental health disorders "),
              pn.pane.Markdown("Data sources: [The Human Development Index (HDI)](https://ourworldindata.org/human-development-index) [The Global Trends in Mental Health Disorder dataset](https://www.kaggle.com/datasets/thedevastator/uncover-global-trends-in-mental-health-disorder?resource=download)")],
    main = [pn.Row(pn.Column(histogram)),
            pn.Row(pn.Column(linear_regression)),
            pn.Row(pn.Column(polynomial_regression)),
            pn.Row(pn.Column(correlation))
           ])
template.show()

### References

##### Data sources:

1. The [Human Development Index](https://ourworldindata.org/human-development-index) was retrieved from Our World in Data (Roser, 2014) and originally published by the United Nations Development Programme. 

2. The [Mental Health dataset](https://www.kaggle.com/datasets/thedevastator/uncover-global-trends-in-mental-health-disorder?resource=download) was retrieved from kaggle by the author Amit.  

Barbalat, G., & Franck, N. (2020). Ecological study of the association between mental illness with human development, income inequalities and unemployment across OECD countries. BMJ open, 10(4), e035055.

Max Roser (2014) - "Human Development Index (HDI)". Published online at OurWorldInData.org. Retrieved from: 'https://ourworldindata.org/human-development-index' [Online Resource]*